# Data Mining Project 2021 - Data Preparation

## Dataset Loading

Loading dataset exported by the Data Understanding phase:

In [2]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import re

from collections import defaultdict
from scipy.stats.stats import pearsonr

pd.options.display.max_columns = None

df = pd.read_csv('DU_matches.csv', sep=',', index_col=0)
df_female = pd.read_csv('DU_female_players.csv', sep=',', index_col=False)
df_male = pd.read_csv('DU_male_players.csv', sep=',', index_col=False)

### fix_based_on_tourney_id(column)

This function fixes NaN values specified in *column* with NON-NaN values of the same *column* in rows where tourney_id is equal. 

In [3]:
def fix_based_on_tourney_id(column):
    edited_rows = 0
    
    # take tourney_ids where column is null and tourney_id is defined
    tourney_ids = list(df[(df[column].isnull()) & (df["tourney_id"].notnull())]["tourney_id"].unique())
    # if there are no rows with these constraints return
    if (len(tourney_ids) == 1 and tourney_ids[0] == np.nan):
        print("no ids with {} == NaN were found.".format(column))
        return edited_rows
    
    # for each tourney_id take the column value if there is at lesta NON null value
    # and replace it in all the rows that have the same tourney_id and a column == NaN
    for tourney_id in tourney_ids:
        tourney_id_column = df[df["tourney_id"] == tourney_id][column].unique()
        if (len(tourney_id_column)!= 1):
            edited_rows += df[(df['tourney_id'] == tourney_id) & (df[column].isnull())].shape[0]
            df.loc[(df['tourney_id'] == tourney_id) & (df[column].isnull()), column] = tourney_id_column[0]
            
    return edited_rows

### fix_based_on_id(id_column, column)

This function fixes NaN values specified in *column* with NON-NaN values of the same *column* in rows where values in *column_id* is equal. 

In [4]:
def fix_based_on_id(column_id, column):
    edited_rows = 0
    
    # take values of column_id where column is null and column_id is defined
    column_id_values = list(df[(df[column].isnull()) & (df[column_id].notnull())][column_id].unique())
    # if there are no rows with these constraints return
    if (len(column_id_values) == 1 and column_id_values[0] == np.nan):
        print("no {} with {} == NaN were found.".format(column_id, column))
        return edited_rows
    
    # for each tourney_id take the column value if there is at lesta NON null value
    # and replace it in all the rows that have the same tourney_id and a column == NaN
    for column_id_value in column_id_values:
        column_value = df[df[column_id] == column_id_value][column].unique()
        if (len(column_value)!= 1):
            edited_rows += df[(df[column_id] == column_id_value) & (df[column].isnull())].shape[0]
            df.loc[(df[column_id] == column_id_value) & (df[column].isnull()), column] = column_value[0]
            
    return edited_rows

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185814 entries, 0 to 186127
Data columns (total 49 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   tourney_id          185759 non-null  object 
 1   tourney_name        185789 non-null  object 
 2   surface             185626 non-null  object 
 3   draw_size           185785 non-null  float64
 4   tourney_level       185785 non-null  object 
 5   tourney_date        185786 non-null  float64
 6   match_num           185787 non-null  float64
 7   winner_id           185759 non-null  float64
 8   winner_entry        25810 non-null   object 
 9   winner_name         185787 non-null  object 
 10  winner_hand         185768 non-null  object 
 11  winner_ht           49299 non-null   float64
 12  winner_ioc          185785 non-null  object 
 13  winner_age          182961 non-null  float64
 14  loser_id            185786 non-null  float64
 15  loser_entry         44087 non-null

# Data Preparation

The Data Preparation phase should prepare the data for further analysis. The main goal that we want to achieve in this section is to try to populate as much data as possible and to delete data rows which containes too many NaN values. 

Once the phase of data cleaning is completed we will create new features for our data set.

### tourney_id attribute
We found 55 missing values, we try to fix them using other rows with the same tourney_name and tourney_date

In [6]:
# get tourney_id from other rows that has same _name and _date
fixable = df[(df["tourney_id"].isnull()) & (df["tourney_name"].notnull()) & (df["tourney_date"].notnull())]
for idx, elem in fixable.iterrows():
    others = df[(df["tourney_id"].notnull()) & (df["tourney_name"]== elem["tourney_name"]) & (df["tourney_date"]==elem["tourney_date"])][:1]
    elem["tourney_id"] = others["tourney_id"].values[0]
    df.loc[idx, 'tourney_id'] = others["tourney_id"].values[0]

df["tourney_id"].isnull().values.sum()

38

In [7]:
df[(df["tourney_date"]== 20171113.0 )]

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,tourney_spectators,tourney_revenue
12023,2017-1059,Taipei,Hard,32.0,C,20171113.0,270.0,205815.0,NaN,Eri Hozumi,R,NaN,JPN,23.737166,201516.0,NaN,Evgeniya Rodina,R,NaN,RUS,28.772074,7-6(2) 6-2,3.0,R32,91.0,3.0,0.0,65.0,45.0,27.0,8.0,10.0,1.0,5.0,3.0,3.0,83.0,53.0,28.0,9.0,10.0,5.0,11.0,173.0,339.0,86.0,752.0,3535.0,638954.72
12024,2017-1059,Taipei,Hard,32.0,C,20171113.0,271.0,211878.0,NaN,Lizette Cabrera,R,NaN,AUS,19.901437,201506.0,NaN,Sara Errani,R,164.0,ITA,30.543463,3-6 7-5 6-2,3.0,R32,127.0,1.0,7.0,94.0,55.0,35.0,19.0,14.0,7.0,11.0,0.0,0.0,104.0,86.0,48.0,5.0,15.0,7.0,13.0,139.0,415.0,145.0,399.0,3535.0,638954.72
12025,2017-1059,Taipei,Hard,32.0,C,20171113.0,272.0,201551.0,NaN,Arantxa Rus,L,180.0,NED,26.918549,203567.0,NaN,Ipek Soylu,R,NaN,TUR,21.579740,7-6(4) 7-5,3.0,R32,101.0,1.0,4.0,66.0,44.0,29.0,12.0,12.0,3.0,6.0,7.0,3.0,79.0,54.0,32.0,11.0,12.0,1.0,5.0,167.0,353.0,194.0,299.0,3535.0,638954.72
12026,2017-1059,Taipei,Hard,32.0,C,20171113.0,273.0,201554.0,Q,Vitalia Diatchenko,R,170.0,RUS,27.282683,201518.0,NaN,Yanina Wickmayer,R,182.0,BEL,28.065708,6-2 6-3,3.0,R32,74.0,4.0,5.0,69.0,38.0,25.0,15.0,9.0,7.0,9.0,6.0,1.0,52.0,36.0,22.0,3.0,8.0,3.0,8.0,176.0,335.0,113.0,561.0,3535.0,638954.72
12027,2017-1059,Taipei,Hard,32.0,C,20171113.0,274.0,211684.0,NaN,Magdalena Frech,R,NaN,POL,19.912389,202684.0,NaN,Lin Zhu,R,NaN,CHN,23.791923,6-4 7-5,3.0,R32,83.0,2.0,1.0,64.0,40.0,29.0,13.0,11.0,3.0,6.0,1.0,3.0,67.0,39.0,28.0,7.0,11.0,4.0,9.0,172.0,341.0,105.0,621.0,3535.0,638954.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186122,NaN,NaN,NaN,32.0,C,20171113.0,NaN,NaN,NaN,Priscilla Hon,NaN,NaN,AUS,19.512663,212026.0,WC,Kwan Yau Ng,NaN,NaN,HKG,NaN,NaN,NaN,Q1,NaN,3.0,4.0,59.0,NaN,23.0,NaN,9.0,NaN,NaN,7.0,4.0,58.0,31.0,21.0,NaN,8.0,8.0,NaN,NaN,NaN,NaN,8.0,NaN,638954.72
186123,NaN,NaN,Hard,32.0,NaN,20171113.0,250.0,NaN,NaN,NaN,NaN,NaN,NaN,26.206708,206075.0,NaN,NaN,NaN,NaN,NaN,26.904860,3-6 6-2 6-3,NaN,NaN,NaN,NaN,1.0,NaN,49.0,38.0,16.0,NaN,7.0,NaN,1.0,3.0,93.0,NaN,NaN,NaN,NaN,7.0,11.0,NaN,322.0,280.0,183.0,NaN,NaN
186124,2017-1059,Taipei,NaN,32.0,C,20171113.0,NaN,NaN,NaN,Veronika Kudermetova,NaN,175.0,NaN,NaN,214562.0,WC,Ingrid Neel,NaN,NaN,USA,NaN,7-6(3) 6-2,NaN,NaN,71.0,10.0,8.0,63.0,NaN,32.0,NaN,10.0,2.0,5.0,NaN,NaN,68.0,NaN,NaN,NaN,NaN,NaN,NaN,255.0,217.0,704.0,NaN,NaN,NaN
186126,NaN,NaN,NaN,32.0,C,20171113.0,NaN,NaN,NaN,Vitalia Diatchenko,NaN,170.0,RUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0-6 6-2 6-4,3.0,NaN,115.0,NaN,8.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,0.0,75.0,NaN,NaN,13.0,NaN,3.0,NaN,176.0,NaN,NaN,NaN,3535.0,NaN


Let's try to integrate also tourney_ravenue

In [8]:
# try to get tourney_id from other rows that has same revenue and (date or name)
fixable = df[(df["tourney_id"].isnull()) & (df["tourney_revenue"].notnull()) & ((df["tourney_date"].notnull()) | (df["tourney_name"].notnull()))]
tmp_df = df
for idx, elem in fixable.iterrows():
    others = df[(df["tourney_id"].notnull()) & (df["tourney_date"].notnull()) & (df["tourney_name"].notnull()) & (df["tourney_revenue"]==elem["tourney_revenue"]) & ((df["tourney_date"]==elem["tourney_date"]) | (df["tourney_name"] == elem["tourney_name"]))][:1]
    elem["tourney_id"] = others["tourney_id"].values[0]
    if (math.isnan(elem["tourney_date"])):
        elem["tourney_date"] = others["tourney_date"].values[0] #fix also missing date
    df.append(elem)

    
df = pd.concat([df, fixable]).drop_duplicates(keep=False)
df["tourney_id"].isnull().values.sum()

27

Showing which are the rows with tourney_id = null

In [9]:
df[(df["tourney_id"].isnull())]

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,tourney_spectators,tourney_revenue
186075,NaN,NaN,NaN,32.0,NaN,NaN,272.0,NaN,NaN,NaN,NaN,NaN,NED,26.918549,203567.0,NaN,NaN,R,NaN,NaN,21.579740,7-6(4) 7-5,3.0,NaN,NaN,NaN,NaN,NaN,44.0,29.0,12.0,NaN,3.0,NaN,7.0,3.0,79.0,NaN,32.0,NaN,NaN,1.0,5.0,NaN,NaN,194.0,NaN,NaN,NaN
186077,NaN,NaN,NaN,NaN,NaN,NaN,274.0,NaN,NaN,Magdalena Frech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,2.0,NaN,NaN,40.0,NaN,13.0,NaN,3.0,6.0,NaN,3.0,67.0,39.0,28.0,NaN,11.0,NaN,9.0,172.0,NaN,105.0,NaN,NaN,NaN
186079,NaN,NaN,Hard,NaN,NaN,NaN,276.0,NaN,NaN,NaN,R,189.0,NaN,NaN,NaN,NaN,Yafan Wang,R,NaN,CHN,NaN,6-7(4) 6-2 6-3,NaN,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,89.0,57.0,37.0,14.0,14.0,NaN,12.0,122.0,501.0,174.0,NaN,3535.0,638954.72
186082,NaN,NaN,NaN,NaN,C,NaN,279.0,NaN,NaN,Jil Belen Teichmann,L,NaN,SUI,20.331280,210931.0,WC,Ching Wen Hsu,R,NaN,TPE,NaN,NaN,3.0,NaN,55.0,0.0,0.0,NaN,NaN,21.0,NaN,NaN,NaN,NaN,0.0,4.0,50.0,NaN,17.0,8.0,NaN,NaN,11.0,NaN,394.0,566.0,NaN,3535.0,NaN
186086,NaN,NaN,Hard,NaN,C,NaN,NaN,NaN,NaN,Marie Bouzkova,R,180.0,NaN,19.315537,201667.0,WC,NaN,R,NaN,NaN,28.043806,6-3 7-5,3.0,R32,NaN,2.0,NaN,67.0,51.0,NaN,NaN,11.0,2.0,5.0,NaN,2.0,60.0,NaN,20.0,11.0,NaN,2.0,NaN,182.0,320.0,333.0,NaN,3535.0,638954.72
186087,NaN,NaN,Hard,32.0,NaN,NaN,NaN,NaN,NaN,Viktorija Golubic,NaN,NaN,SUI,25.075975,202693.0,NaN,NaN,R,NaN,TPE,NaN,NaN,NaN,NaN,NaN,0.0,5.0,72.0,NaN,32.0,10.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,20.0,NaN,10.0,NaN,NaN,116.0,515.0,NaN,138.0,3535.0,NaN
186091,NaN,NaN,NaN,NaN,C,20171113.0,288.0,NaN,NaN,NaN,R,175.0,NaN,20.555784,211684.0,NaN,NaN,R,NaN,POL,19.912389,1-6 6-3 6-4,NaN,R16,NaN,NaN,5.0,NaN,NaN,NaN,12.0,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,11.0,NaN,5.0,11.0,255.0,217.0,172.0,NaN,3535.0,NaN
186093,NaN,Taipei,Hard,NaN,C,NaN,290.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,203552.0,NaN,Risa Ozaki,R,NaN,NaN,23.594798,6-4 6-2,3.0,R16,94.0,5.0,NaN,NaN,NaN,NaN,18.0,9.0,NaN,5.0,3.0,0.0,NaN,NaN,25.0,17.0,9.0,NaN,NaN,NaN,394.0,NaN,NaN,NaN,NaN
186094,NaN,Taipei,Hard,32.0,NaN,NaN,291.0,NaN,WC,Belinda Bencic,R,NaN,SUI,NaN,211880.0,Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,4.0,61.0,NaN,26.0,14.0,10.0,2.0,4.0,5.0,2.0,68.0,35.0,NaN,13.0,NaN,NaN,10.0,120.0,502.0,231.0,NaN,NaN,NaN
186096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R,NaN,SUI,NaN,214544.0,NaN,Aryna Sabalenka,R,182.0,BLR,NaN,7-5 6-3,NaN,R16,NaN,NaN,1.0,71.0,48.0,NaN,NaN,10.0,4.0,7.0,NaN,6.0,NaN,NaN,21.0,9.0,11.0,NaN,7.0,NaN,NaN,NaN,NaN,3535.0,638954.72


**We can not infer with 100 % degree of certainity the tourney attributes in these 27 rows.
We decided to keep them, even if many missing values are present, to make future analysis (i.e. winner_name is not null in all the rows so it can be used to incremente the number of won matches by that players.)**

### Checking tourney_name

In [10]:
df[(df["tourney_name"].isnull())].shape[0]

20

Trying to find the tourney_name based on the tourney_id

In [11]:
rows = fix_based_on_id('tourney_id', 'tourney_name')
rows

0

Trying to find the tourney_name based on the tourney_date and revenue

In [12]:
df[(df["tourney_name"].isnull()) & (df["tourney_date"].notnull()) & (df["tourney_revenue"].notnull())]

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,tourney_spectators,tourney_revenue


There aren't enough information to replace the tourney_name.

### Checking tourney_date

In [13]:
df[(df["tourney_date"].isnull())].shape[0]

22

Trying to find the tourney_date based on the tourney_id

In [14]:
rows = fix_based_on_id('tourney_id', 'tourney_date')
rows

0

Trying to find the tourney_date based on the tourney_name and tourney_revenue

In [15]:
df[(df['tourney_date'].isnull()) & (df['tourney_name'].notnull()) & (df['tourney_revenue'].notnull())]

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,tourney_spectators,tourney_revenue


### Checking surface

Let's check if there is some rows with surface that is NaN and with tourney_id defined

In [16]:
df[(df["surface"].isnull()) & (df["tourney_id"].notnull())].shape[0]

171

In [17]:
rows = fix_based_on_id('tourney_id', 'surface')
rows

9

9 rows fixed.

### Checking draw_size

Let's check if there is some rows with draw_size that is NaN and with tourney_id defined

In [18]:
df[(df["draw_size"].isnull()) & (df["tourney_id"].notnull())].shape[0]

10

In [19]:
rows = fix_based_on_id('tourney_id', 'draw_size')
rows

10

### Checking tourney_spectators

Checking if there are row in which there is the tourney_id bu not the tourney_spectators

In [20]:
df[(df["tourney_spectators"].isnull()) & (df["tourney_id"].notnull())].shape[0]

8

In [21]:
rows = fix_based_on_id('tourney_id', 'tourney_spectators')
rows

8

Done

### Checking tourney_revenue

Checking if there are row in which there is the tourney_id but not the tourney_revenue

In [22]:
df[(df["tourney_revenue"].isnull()) & (df["tourney_id"].notnull())].shape[0]

7

In [23]:
rows = fix_based_on_id('tourney_id', 'tourney_revenue')
rows

7

Done

Before checking if winners and losers' attributes can be fixed through the winner or loser ids, I want to check that there is different winner/loser names with the same id.

In [24]:
not_unique_winner_ids = []
not_unique_loser_ids = []

print ("=============== WINNERS ===============")
for id in df['winner_id'].dropna().unique().tolist():
    if len(df[df['winner_id'] == id]['winner_name'].dropna().unique().tolist()) > 1:
        print (id, df[df['winner_id'] == id]['winner_name'].dropna().unique())
        not_unique_winner_ids.append(id)
        
print ("=============== LOSERS ===============")
for id in df['loser_id'].dropna().unique().tolist():
    if len(df[df['loser_id'] == id]['loser_name'].dropna().unique().tolist()) > 1:
        not_unique_loser_ids.append(id)
        print (id, df[df['loser_id'] == id]['loser_name'].dropna().unique())

=============== WINNERS ===============
202434.0 ['Aleksandra Krunic' 'Ricardo Ortiz']
202421.0 ['Alison Van Uytvanck' 'George Houghton']
206173.0 ['Jannik Sinner' 'Beatrice Lombardo']
202475.0 ['Philip Henning' 'Jasmina Tinjic']
202426.0 ['Julia Boserup' 'Matt Halpin']
202420.0 ['An Sophie Mestach' 'Aidan Mchugh']
206091.0 ['Yolande Leacock' 'Yan Bondarevskiy']
206171.0 ['Alice Bacquie' 'Barnaby Smith']
206137.0 ['Oleksandra Piskun' 'Andrej Glvac']
202359.0 ['Chin Wei Chan' 'Marvin Moeller']
216569.0 ['Zeynep  Sena Sarioglan' 'Zeynep Sena Sarioglan']
202349.0 ['Rushmi Chakravarthi' 'Kai Lemstra']
208147.0 ['Alexandre Aubriot' 'Pamela Duran Vinueza']
=============== LOSERS ===============
202428.0 ['Kiki Bertens' 'Joao Pedro Alcantara']
202434.0 ['Aleksandra Krunic' 'Ricardo Ortiz']
202429.0 ['Camila Giorgi' 'David Lucas Ambrozic']
202421.0 ['Alison Van Uytvanck' 'George Houghton']
200033.0 ['Serena Williams' 'Kushaan Nath']
202437.0 ['Anastasia Grymalska' 'Yang Liu']
202440.0 ['Paula 

There are different players who have the same player_id. This is a problem in case we want to fix some missing values based on players ids. **We decided to use the name as unique identifier for the players.**

### Checking winner_name and loser_name

In [25]:
df[(df["winner_name"].isnull()) & (df["winner_id"].notnull())].shape[0]

0

There aren't enough information to replace the winner_name. (winner_id is NaN for each row in which winner_name is NaN)

In [26]:
df[(df["loser_name"].isnull()) & (df["loser_id"].notnull())].shape[0]

12

some loser_name maybe can be fixed. We need to consider that we have different players with same id so we need to figure out if a lsoer_id is only one name associated or not: is this is true we can replace the name.

In [27]:
loser_ids = list(df[(df['loser_name'].isnull()) & (df["loser_id"].notnull())]["loser_id"].unique())
for loser_id in loser_ids:
    if loser_id in not_unique_loser_ids:
        print (loser_id)

Ok we can fix all of this missing values.

In [28]:
rows = fix_based_on_id('loser_id', 'loser_name')
rows

12

All loser_name fixed.

#### Checking for alphanumeric codes in names

In [29]:
regex = r'\b\s[a-zA-Z]*[0-9]+[a-zA-Z]*\b\s*' # this regex find all the word that has the code pattern
findings = []

not_null_winner = df[df["winner_name"].notnull()]
for elem in not_null_winner["winner_name"].unique():
    if bool(re.search(r'\d', elem)):
        findings.append(elem)

not_null_loser = df[df["loser_name"].notnull()]
for elem in not_null_loser["loser_name"].unique():
    if bool(re.search(r'\d', elem)):
        findings.append(elem)

del not_null_loser
del not_null_winner
np.array(findings) # pretty print

array(['Alejandro Gomez Gb42', 'Ekaterina Makarova 1996',
       'Alejandro Gomez Gb42', 'Michael Grant Gd13',
       'Ekaterina Makarova 1996'], dtype='<U23')

Fixing them

In [30]:
#removing codes from names with rege
regex = r'(.*?)\b\s[a-zA-Z]*[0-9]+[a-zA-Z]*\b\s*'
df["winner_name"].replace(to_replace=regex, value=r"\1", regex=True, inplace=True)
df["loser_name"].replace(to_replace=regex, value=r"\1", regex=True, inplace=True)

### Checking winner and loser IOC

In [31]:
df[(df["winner_ioc"].isnull()) & (df["winner_id"].notnull())].shape[0]

0

No winner_ioc can be fixed by using winner_id

In [32]:
df[(df["winner_ioc"].isnull()) & (df["winner_name"].notnull())].shape[0]

11

Maybe some winner ioc can be fixed by using winner_name. As we mentioned before, we are using winner_name as unique identifier of the players

In [33]:
rows = fix_based_on_id('winner_name', 'winner_ioc')
rows

11

Super! All the 11 rows have been fixed.

In [34]:
df[(df["loser_ioc"].isnull()) & (df["loser_id"].notnull())].shape[0]

10

some loser_ioc maybe can be fixed. But we need to check that these ids are not the one that are not unique.

In [35]:
loser_ids = list(df[(df['loser_name'].isnull()) & (df["loser_id"].notnull())]["loser_id"].unique())
for loser_id in loser_ids:
    if loser_id in not_unique_loser_ids:
        print (loser_id)

In [36]:
rows = fix_based_on_id('loser_id', 'loser_ioc')
rows

10

Let's see if also by usign loser_name some loser_ioc can be fixed.

In [37]:
df[(df["loser_ioc"].isnull()) & (df["loser_name"].notnull())].shape[0]

3

In [38]:
rows = fix_based_on_id('loser_name', 'loser_ioc')
rows

3

### Checking winner and loser height

In [39]:
df[df['winner_ht'] < 150][["winner_ht", "winner_name"]].value_counts()

winner_ht  winner_name      
2.0        Kamilla Rakhimova    81
145.0      Ilija Vucic           9
dtype: int64

Fix these two through data integration:
- Ilija Vucic 188.0 (https://www.sofascore.com/team/tennis/vucic-ilija/79519)
- Kamilla Rakhimova 174.0 (Google)

In [40]:
df.loc[df['winner_name']== 'Ilija Vucic', 'winner_ht'] = 188.0
df.loc[df['loser_name']== 'Ilija Vucic', 'loser_ht'] = 188.0

df.loc[df['winner_name']== 'Kamilla Rakhimova', 'winner_ht'] = 174.0
df.loc[df['loser_name']== 'Kamilla Rakhimova', 'loser_ht'] = 174.0

Checking if there are some players who have not height in all their matches rows.

In [41]:
names = pd.concat([df['winner_name'], df['loser_name']]).dropna().unique().tolist()
len(names)

10103

Of these 10104 unique names only 541 have the heights. 

**They are very fews and we do not think they are enough to create meaningfull statistical features. For this reason we thing that we can not replace missing values through statistical features like mean or mode.** 

### Checking winner and loser age

In [42]:
df[df['winner_age'] > 65][["winner_age", "winner_name", "tourney_date"]]

,winner_age,winner_name,tourney_date
0,95.0,Kei Nishikori,20181231.0
322,95.0,Daniil Medvedev,20190114.0


We decided to replace these wrong values by using data integration:
- Kei Nishikori date of birth: 29/12/1989 
- Daniil Medvedev date of birth: 11/02/1996 

In [43]:
df.loc[(df['winner_name'] == 'Kei Nishikori'), 'winner_age'] = 29
df.loc[(df['winner_name'] == 'Daniil Medvedev'), 'winner_age'] = 22

In [44]:
df[df['loser_age'] > 65][["loser_age", "loser_name", "tourney_date"]]

,loser_age,loser_name,tourney_date
176662,74.294319,Gail Falkenberg,20210503.0
176943,74.485969,Gail Falkenberg,20210712.0


Gail Falkenberg is 74 years old. Data is correct.

### Checking male and female dataset

Let's see if we have some codes in the names

In [45]:
#find codes in names with regex
regex = r'\b\s[a-zA-Z]*[0-9]+[a-zA-Z]*\b\s*' # this regex find all the word that has the code pattern
findings = []

df_male = df_male[(df_male["name"].notnull()) & (df_male["surname"].notnull())]
for elem in df_male["name"]:
    if bool(re.search(r'\d', elem)):
        findings.append(elem)
        
for elem in df_male["surname"]:
    if bool(re.search(r'\d', elem)):
        findings.append(elem)
        
np.array(findings) #to print it better

array(['Garcia G003', 'Gonzalez G162', 'Casey C100', 'Phillips P239',
       'Jonsson J092', 'Gutierrez G230', 'Lindholm L174',
       'Turturici T225', 'Brown B395', 'Robinson R261',
       'Strengberger S597', 'Parun P991', 'Gonzalez G419',
       'Phillips Pd13', 'Deleuran Skjold Db37', 'Gomez Gb42',
       'Kocyla Kd20', 'Weise W055', 'Ukon U004', 'Perez P114',
       'Pueski P199', 'Pueski P223', 'Robinson R263', 'Wright W096',
       'Parun P071', 'Khalfan K735', 'Moore2', 'Smith 2', 'Day 3D',
       'Symington 3D', 'Post 3Rd', 'Leroy 2', 'Bergevi Bl61',
       'Grant Gd13', 'Hernandez Hb01', 'Perez P727', 'Lloyd L770',
       'Kovacs Kb22', 'Fernandes F729', 'Lindstrom2', 'Ul Haq H325',
       'Ukon U033', 'Ul Haq U022', 'Goransson G423', 'Antelo A325',
       'Roman R410', 'Machkamov M741', 'Williams W303', 'Robert Rb04',
       'Fernandez F929', 'Hassan Hb88', 'Reyes Rc60', '1'], dtype='<U20')

In [46]:
#removing codes from names with rege
regex = r'(.*?)\b\s[a-zA-Z]*[0-9]+[a-zA-Z]*\b\s*'
df_male["surname"].replace(to_replace=regex, value=r"\1", regex=True, inplace=True)
#some has just a number at the end
regex = r'(.*?)\s*[0-9]+\s*'
df_male["surname"].replace(to_replace=regex, value=r"\1", regex=True, inplace=True)

Fixed. Let's do this with females

In [47]:
#find codes in names with regex
regex = r'\b\s[a-zA-Z]*[0-9]+[a-zA-Z]*\b\s*' # this regex find all the word that has the code pattern
findings = []

df_female = df_female[(df_female["name"].notnull()) & (df_female["surname"].notnull())]
for elem in df_female["name"]:
    if bool(re.search(r'\d', elem)):
        findings.append(elem)
        
for elem in df_female["surname"]:
    if bool(re.search(r'\d', elem)):
        findings.append(elem)
        
np.array(findings) #to print it better

array(['Makarova 1996'], dtype='<U13')

In [48]:
#removing numbers at the end
regex = r'(.*?)\s*[0-9]+\s*'
df_female["surname"].replace(to_replace=regex, value=r"\1", regex=True, inplace=True)

#### Removing Mr/Mrs from names

In [49]:
#removing Mrs
regex = r'(\s*Mrs\s*)(.*)'
df_female["name"].replace(to_replace=regex, value=r"\1", regex=True, inplace=True)
df_female["surname"].replace(to_replace=regex, value=r"\1", regex=True, inplace=True)

#removing Mr
regex = r'(\s*Mr\s*)(.*)'
df_male["name"].replace(to_replace=regex, value=r"\1", regex=True, inplace=True)
df_male["surname"].replace(to_replace=regex, value=r"\1", regex=True, inplace=True)

#### Checking if we can recover some shortned names/surnames for females and males

In [50]:
single_letter_f = df_female[(df_female["name"].notnull()) & (df_female["surname"].notnull()) & ((df_female["name"].str.len() == 1) | (df_female["surname"].str.len() == 1))]
single_letter_f

,Unnamed: 0,name,surname
137,138,L,Griffiths
182,183,E,Staples
206,207,L,Cameron
3600,3601,G,Wilson
3659,3660,J,Tobin
...,...,...,...
45589,46100,M,Kiss
45604,46115,C,Bushell
45605,46116,E,Bushell
45648,46160,B,Wallis


In [51]:
#try to find duplicates (es. name is with 1 letter but exists other row with full name)
for idx, elem in single_letter_f.iterrows():
    others = df_female[(df_female["name"][0]==elem["name"]) & (df_female["surname"]==elem["surname"])]
    if len(others)>0:
        print(elem["name"] + " " + elem["surname"])
    others = df_female[(df_female["name"]==elem["name"]) & (df_female["surname"][0]==elem["surname"])]
    if len(others)>0:
        print(elem["name"] + " " + elem["surname"])
print("Done")

Done


ok, no duplicates with name. Let's do it for males

In [52]:
single_letter_m = df_male[(df_male["name"].notnull()) & (df_male["surname"].notnull()) & ((df_male["name"].str.len() == 1) | (df_male["surname"].str.len() == 1))]
single_letter_m

,Unnamed: 0,name,surname
4705,4707,G,Jones
6417,6419,A,Benson
6418,6420,A,Escofet
6419,6421,A,Hall
6420,6422,A,Noffat
...,...,...,...
54355,54873,N,Gillasby
54356,54874,J,Frederick
54357,54875,J,Fannin
54358,54876,C,Whittle


In [53]:
#try to find duplicates (es. name is with 1 letter but exists other row with full name)
for idx, elem in single_letter_m.iterrows():
    others = df_male[(df_male["name"][0]==elem["name"]) & (df_male["surname"]==elem["surname"])]
    if len(others)>0:
        print(elem["name"] + " " + elem["surname"])
    others = df_male[(df_male["name"]==elem["name"]) & (df_male["surname"][0]==elem["surname"])]
    if len(others)>0:
        print(elem["name"] + " " + elem["surname"])
print("Done")

Done


ok, no duplicates with name

### Removing useless rows

All the rows in which neither winner_name and loseer_name are defined can be droped in our opinion. We want to create players profiles and rows in which this two attributes ar NaN are useless.

In [54]:
df[df["winner_name"].isnull() & df["loser_name"].isnull()]

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,tourney_spectators,tourney_revenue
186073,2017-1059,Taipei,Hard,32.0,NaN,20171113.0,270.0,NaN,NaN,NaN,R,NaN,JPN,23.737166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,R32,NaN,3.0,0.0,65.0,45.0,27.0,8.0,NaN,NaN,5.0,NaN,3.0,NaN,53.0,NaN,NaN,NaN,NaN,NaN,173.0,NaN,86.0,NaN,3535.0,638954.72
186108,NaN,NaN,NaN,NaN,NaN,20171113.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.289528,NaN,NaN,NaN,NaN,NaN,KOR,25.355236,NaN,3.0,Q1,91.0,NaN,NaN,79.0,49.0,NaN,11.0,NaN,NaN,8.0,0.0,4.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186110,NaN,Taipei,Hard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WC,NaN,U,NaN,NaN,20.731006,NaN,NaN,Q1,NaN,NaN,4.0,NaN,30.0,23.0,NaN,NaN,NaN,3.0,7.0,NaN,58.0,31.0,NaN,8.0,8.0,NaN,NaN,231.0,NaN,NaN,8.0,NaN,NaN
186113,2017-1059,Taipei,Hard,32.0,NaN,20171113.0,NaN,NaN,NaN,NaN,R,NaN,NaN,26.642026,NaN,NaN,NaN,R,NaN,NaN,23.192334,NaN,3.0,NaN,70.0,NaN,NaN,NaN,NaN,NaN,14.0,8.0,NaN,NaN,1.0,NaN,50.0,31.0,15.0,NaN,8.0,5.0,11.0,NaN,228.0,196.0,299.0,3535.0,638954.72


In [55]:
df.drop(df[df["winner_name"].isnull() & df["loser_name"].isnull()].index, inplace=True)

## New features creation

### Creating GENDER feature in matches

Let's start creating FULLNAME and GENDER in male/female datasets

In [56]:
# add fullname and gender feature to male and female and genders to matches df
df_male["fullname"] = df_male["name"] + " " + df_male["surname"]
df_male["gender"] = "M"
df_female["fullname"] = df_female["name"] + " " + df_female["surname"]
df_female["gender"] = "F"
print("Fullname and Gender features created!")

Fullname and Gender features created!


Merging the two datasets to have a players dataset

In [57]:
# creating players dataset
df_players = pd.concat([df_male, df_female])
df_players.drop(columns=["name", "surname", "Unnamed: 0"], inplace=True)
df_players.drop_duplicates(inplace=True)
df_players = df_players[df_players["fullname"].notnull()]

Checking if we have some player that are both M and F 

In [58]:
# checking non-binary
dropped = df_players.drop(columns="gender")
mask = dropped.duplicated(subset=['fullname'])
nonbinary = dropped.loc[mask]
nonbinary.shape[0]

64

64 players are both M and F. We need to find their real gender.

In [59]:
# remove non binary players from df_players
defined_players = df_players.loc[~mask]
for elem in nonbinary["fullname"].values:
    defined_players = defined_players[defined_players["fullname"] != elem]

# merging with matches to try getting the gender from opponent
df = pd.merge(df, defined_players, how="left", left_on="winner_name", right_on="fullname")
df.rename(columns={"fullname":"fullname_w", "gender": "gender_w"}, inplace=True)
df = pd.merge(df, defined_players, how="left", left_on="loser_name", right_on="fullname")
df.rename(columns={"fullname":"fullname_l", "gender": "gender_l"}, inplace=True)

#Fixing non-binary rows in df_player
done = (df["gender_w"].notnull() | df["gender_l"].notnull())
for elem in nonbinary["fullname"].values:
    fixable = df[done & (df["winner_name"] == elem)]
    if len(fixable)>0:
        choice = fixable["gender_w"].value_counts()
        if len(choice) > 0: #nice, we have found the right gender (the only one or the more frequent)
            nonbinary.loc[nonbinary["fullname"] == elem, "gender"] = fixable.iloc[0]["gender_w"]
        else: #no match, let's try with losers
            choice = fixable["gender_l"].value_counts()
            if len(choice) > 0: #nice, we have found the right gender (the only one or the more frequent)
                nonbinary.loc[nonbinary["fullname"] == elem, "gender"] = fixable.iloc[0]["gender_l"]

df_players = pd.concat([defined_players, nonbinary[nonbinary["gender"].notnull()]])

#cleaning df
df.drop(columns=["fullname_w", "fullname_l"], inplace=True)

/usr/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


**This warning is a false positive and the code works as intended!**

Checking if there are match with different genders opponents

In [60]:
df[(df["gender_w"] != df["gender_l"]) & (df["gender_w"].notnull()) & (df["gender_l"].notnull()) ]

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,tourney_spectators,tourney_revenue,gender_w,gender_l


Perfect.

### Creating SEASON feature

In [61]:
tot_str = str(df.shape[0])
all_dates = df[df["tourney_date"].notnull()]["tourney_date"]
all_dates.drop_duplicates(inplace=True)

for idx, elem in all_dates.iteritems():
    month = int(str(elem)[4:6])
    if (month >= 10 or month <= 3):
        month = "Winter"
    else:
        month = "Summer"
    df.loc[df["tourney_date"] == elem, "season"] = month
    print('Progress: '+ str(idx)+"/"+tot_str, end='\r')

/usr/lib/python3.9/site-packages/pandas/core/series.py:2136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(result)


**This warning is a false positive and the code works as intended!**

### Gender and Fullnames fixing

In this section we will try to fix all the names that didn't match, players that do not have a gender.

In [62]:
no_gender = df[df["gender_w"].isnull()]["winner_name"].value_counts().to_dict()

In [63]:
gender_tofind = []
for fullname in no_gender:
    gender_tofind.append(fullname)

In [64]:
gender_tofind.sort()

In [65]:
gender_tofind

['Alexandar Lazov',
 'Alona Fomina',
 'Andres Artunedo Martinavarro',
 'Antoine Hoang',
 'Austin Smith',
 'Ben Patael',
 'Botic van de Zandschulp',
 "Christopher O'Connell",
 'Cristian Garin',
 'Daniel Elahi Galan',
 'Daniel Munoz de la Nava',
 "David O'Hare",
 'Di Zhao',
 'Diego Schwartzman',
 'Evgenii Tiurnev',
 'Frances Tiafoe',
 'Franko Skugor',
 'Holger Rune',
 'J.J. Wolf',
 'Jo-Wilfried Tsonga',
 'Joao Menezes',
 'Jonathan Mridha',
 'Juan Martin del Potro',
 'Juan Pablo Varillas',
 'Jurabek Karimov',
 'Khumoun Sultanov',
 'Kuan Yi Lee',
 'Lloyd Harris',
 'Mackenzie McDonald',
 'Pedro Martinez',
 'Sam Groth',
 'Stan Wawrinka',
 'Taylor Fritz',
 'Yi Liu',
 'Zeynep  Sena Sarioglan',
 'Ziyu Zhang']

In [66]:
def replace_gender(name, gender):
    df.loc[df["loser_name"] == name, "gender_l"] = gender
    df.loc[df["winner_name"] == name, "gender_w"] = gender

In [67]:
def replace_name(old_name, name, gender):
    df.loc[df["loser_name"] == old_name, "gender_l"] = gender
    df.loc[df["winner_name"] == old_name, "gender_w"] = gender
    df.loc[df["winner_name"] == old_name, "winner_name"] = name
    df.loc[df["loser_name"] == old_name, "loser_name"] = name

In [68]:
# Alexandar Lazov
df_male[df_male["name"] == "Alexandar"]

,Unnamed: 0,name,surname,fullname,gender
24128,24283,Alexandar,Kasarov,Alexandar Kasarov,M
45022,45281,Alexandar,Lazarov,Alexandar Lazarov,M
51690,52159,Alexandar,Yordanov,Alexandar Yordanov,M


name 'Alexandar Lazov' is probably a wrong value into the matches dataset, the correct name is 'Alexandar Lazarov'

In [69]:
replace_name("Alexandar Lazov","Alexandar Lazarov","M")

In [70]:
# Alona Fomina
df_female[df_female["surname"] == "Fomina"].value_counts()

Unnamed: 0  name   surname  fullname      gender
6404        Alena  Fomina   Alena Fomina  F         1
dtype: int64

name 'Alona Fomina' is probably a wrong value into the matches dataset, the correct name is 'Alena Fomina'

In [71]:
# Alena Fomina
replace_name("Alona Fomina","Alena Fomina","F")

In [72]:
# 'Antoine Hoang',

In [73]:
replace_gender("Antoine Hoang","M")

'Andres Artunedo Martinavarro' is not into the players dataset

In [74]:
replace_gender("Andres Artunedo Martinavarro","M")

In [75]:
# Austin Smith
df_male[df_male["surname"]== "Smith"].value_counts()

Unnamed: 0  name       surname  fullname         gender
125         Stan       Smith    Stan Smith       M         1
41720       Alexandre  Smith    Alexandre Smith  M         1
41592       Lewis      Smith    Lewis Smith      M         1
40722       Zachery    Smith    Zachery Smith    M         1
40058       Trevor     Smith    Trevor Smith     M         1
                                                          ..
12395       Douglas    Smith    Douglas Smith    M         1
12240       Wilfred H  Smith    Wilfred H Smith  M         1
12151       A          Smith    A Smith          M         1
12022       J          Smith    J Smith          M         1
54815       Barry      Smith    Barry Smith      M         1
Length: 115, dtype: int64

there is an A Smith, probably is Austin Smith

In [76]:
replace_gender("Austin Smith","M")

In [77]:
#Ben Patael
df_male[df_male["surname"] == "Patael"].to_dict()
# esiste un Ben Patael ma non è nel dataset dei players

{'Unnamed: 0': {}, 'name': {}, 'surname': {}, 'fullname': {}, 'gender': {}}

In [78]:
replace_gender("Ben Patael","M")

In [79]:
# Botic van de Zandschulp'
df_male[df_male["name"] == "Botic"]

,Unnamed: 0,name,surname,fullname,gender
22136,22282,Botic,Van De Zandschulp,Botic Van De Zandschulp,M


In [80]:
replace_gender("Botic van de Zandschulp","M")

In [81]:
# Christopher O'Connell
df_male[df_male["surname"] == "Oconnell"].value_counts()

Unnamed: 0  name         surname   fullname              gender
6316        Christopher  Oconnell  Christopher Oconnell  M         1
15446       George J     Oconnell  George J Oconnell     M         1
19685       David        Oconnell  David Oconnell        M         1
37819       William      Oconnell  William Oconnell      M         1
50779       Fergus       Oconnell  Fergus Oconnell       M         1
51770       Benjamin     Oconnell  Benjamin Oconnell     M         1
dtype: int64

Christopher  Oconnell  is written in a wrong way, is the same person. The correct fullname is Christopher O'Connell (the wrong value is into the male players dataset)

In [82]:
replace_gender("Christopher O'Connell","M")

In [83]:
# Cristian Garin
df_male[df_male["surname"] == "Garin"]

,Unnamed: 0,name,surname,fullname,gender
6409,6411,Christian,Garin,Christian Garin,M


The correct fullname is Cristian Garin, not Christian Garin (is wrong into the male players dataset)

In [84]:
replace_gender("Cristian Garin","M")

In [85]:
# Daniel Elahi Galan
p = df_male[df_male["name"] == "Daniel"]["surname"].value_counts().to_dict()

No found... need to integrate and insert the gender

In [86]:
replace_gender("Daniel Elahi Galan","M")

In [87]:
# Daniel Munoz de la Nava 
df_male[df_male["surname"] == "Munoz De La Nava"]

,Unnamed: 0,name,surname,fullname,gender
3923,3925,Daniel,Munoz De La Nava,Daniel Munoz De La Nava,M


found, but the fullname into the matches dataset doesn't has capital letter --> need to fix dataset matches

In [88]:
replace_gender("Daniel Munoz de la Nava","M")

In [89]:
#  "David O'Hare",
df_male[df_male["surname"] == "Ohare"]

,Unnamed: 0,name,surname,fullname,gender
10449,10459,David,Ohare,David Ohare,M


the surname into the players dataset is wrong, should be O'Hare

In [90]:
replace_gender("David O'Hare","M")

In [91]:
# Di Zhao
df_players[df_players["fullname"] == "Di Zhao"].value_counts()

fullname  gender
Di Zhao   F         1
dtype: int64

In [92]:
replace_gender("Di Zhao","F")

In [93]:
# Diego Schwartzman
df_male[df_male["surname"] == "Schwartzman"].value_counts()


Unnamed: 0  name             surname      fullname                     gender
6028        Diego Sebastian  Schwartzman  Diego Sebastian Schwartzman  M         1
dtype: int64

real name is Diego Sebastian Schwartzman, need to fix the dataset matches with the right fullname

In [94]:
replace_name("Diego Schwartzman","Diego Sebastian Schwartzman","M")

In [95]:
 # 'Evgenii Tiurnev',
a = df_male[df_male["name"] == "Evgenii"]
b = df_male[df_male["surname"] == "Tiurnev"]
a
b

,Unnamed: 0,name,surname,fullname,gender


is not into the players dataset, maybe we could do data integration

In [96]:
replace_gender("Evgenii Tiurnev","M")

In [97]:
# Frances Tiafoe
df_male[df_male["surname"] == "Tiafoe"]

,Unnamed: 0,name,surname,fullname,gender
26005,26191,Francis,Tiafoe,Francis Tiafoe,M


correct name is Frances Tiafoe, need to fix the players dataset

In [98]:
replace_gender("Frances Tiafoe","M")

In [99]:
# 'Franko Skugor'
df_male[df_male["surname"] == "Skugor"]

,Unnamed: 0,name,surname,fullname,gender
5006,5008,Franco,Skugor,Franco Skugor,M


correct name is Franko Skugor, need to fix the matches dataset

In [100]:
replace_gender("Franko Skugor","M")

In [101]:
# 'Holger Rune'
df_male[df_male["surname"] == "Rune"]

,Unnamed: 0,name,surname,fullname,gender
52591,53063,Holger Vitus Nodskov,Rune,Holger Vitus Nodskov Rune,M


the correct fullname is Holger Nodskov, need to fix the players dataset

In [102]:
replace_gender("Holger Rune","M")

In [103]:
# J.J. Wolf
df_male[df_male["surname"] == "Wolf"].value_counts()

Unnamed: 0  name          surname  fullname           gender
1191        Mike          Wolf     Mike Wolf          M         1
5338        Matthias      Wolf     Matthias Wolf      M         1
9389        Jeff          Wolf     Jeff Wolf          M         1
15444       Harry F       Wolf     Harry F Wolf       M         1
23507       Michael       Wolf     Michael Wolf       M         1
24947       David         Wolf     David Wolf         M         1
30930       Sebastian     Wolf     Sebastian Wolf     M         1
45704       Jeffrey John  Wolf     Jeffrey John Wolf  M         1
dtype: int64

the correct name is Jeffrey John, need to fix the matches dataset 

In [104]:
replace_name("J.J. Wolf","Jeffrey John Wolf","M")

In [105]:
# Jo-Wilfried Tsonga
df_male[df_male["surname"] == "Tsonga"].value_counts()

Unnamed: 0  name         surname  fullname            gender
4540        Jo Wilfried  Tsonga   Jo Wilfried Tsonga  M         1
dtype: int64

need to decide if we want to drop the "-"

In [106]:
replace_gender("Jo-Wilfried Tsonga","M")

In [107]:
# Joao Menezes
df_male[df_male["surname"] == "Menezes"].value_counts()

Unnamed: 0  name     surname  fullname         gender
948         Mauro    Menezes  Mauro Menezes    M         1
28581       Gustavo  Menezes  Gustavo Menezes  M         1
dtype: int64

the player exist but is not into the male players dataset, maybe we could do integration

In [108]:
replace_gender("Joao Menezes","M")

In [109]:
# 'Juan Martin del Potro'
df_male[df_male["name"] == "Juan"].value_counts()

Unnamed: 0  name  surname            fullname                gender
63          Juan  Gisbert            Juan Gisbert            M         1
39356       Juan  Orendain           Juan Orendain           M         1
37731       Juan  Jorgui             Juan Jorgui             M         1
37835       Juan  Barrera            Juan Barrera            M         1
38111       Juan  Moreiras           Juan Moreiras           M         1
                                                                      ..
14848       Juan  Aguirre            Juan Aguirre            M         1
14743       Juan  Weiss              Juan Weiss              M         1
11683       Juan  Berrido Fernandez  Juan Berrido Fernandez  M         1
11514       Juan  Mejia              Juan Mejia              M         1
54770       Juan  Carlos Arroyo      Juan Carlos Arroyo      M         1
Length: 136, dtype: int64

there is a Juan Martin, but the correct fullname is "Juan Martin del Potro",  need to fix the male players dataset

In [110]:
replace_gender("Juan Martin del Potro","M")

In [111]:
# 'Juan Pablo Varillas'
print(df_male[df_male["name"] == "Juan"]["surname"].unique())

['Gisbert' 'Nunez' 'Farrow' 'Avendano' 'Aguilera' 'Pino' 'Rios' 'Lavalle'
 'Olivert' 'Garat' 'Gisbert Jr' 'Balcells' 'Morgenstern' 'Camilo Gamboa'
 'Giner' 'Ferrer' 'Monaco' 'De Armas' 'Beaus Barquin' 'Vazquez Valenzuela'
 'Lizariturry' 'Fiesta' 'Boyanovich' 'Jimenez' 'Martin Bra' 'Rivera'
 'Couder' 'Diaz' 'Hernandez' 'Herrera' 'Leal' 'Ajuanqa' 'Torralbo'
 'Barragan' 'Jurado' 'Lopez Vico' 'Mejia' 'Berrido Fernandez' 'Weiss'
 'Aguirre' 'Notz' 'Arredondo' 'Perianez Llopis' 'Torres Palacios'
 'Pardo Moran' 'Alfaro' 'Bolona' 'Fernandez' 'Rodriguez' 'Obeid' 'Rocha'
 'Reitz' 'Montes' 'Lomba Diego' 'Gavilan Puerto' 'Ayala'
 'Barrio Gutierrez' 'Ordonez' 'De Beer' 'Guerrero' 'Sanchez Silva'
 'Martin Adalia' 'Lovado' 'Escobar' 'Millanao' 'Fornell' 'Borras Meijides'
 'England' 'Duke' 'Osete Moreno' 'Centella Perez' 'Carranza Novoa' 'Erro'
 'Espinel' 'Borba' 'Ortiz Couder' 'Rivadeneira' 'Espinosa' 'Delvalle'
 'Madrid Segarra' 'Rivera Cabeo' 'Riviere Lemmel' 'Borras Meijide'
 'Gonzalez Sanchez' 'Re

he is not into the male players dataset, but he exists in reality, we cloud do data integrartion 

In [112]:
replace_gender("Juan Pablo Varillas","M")

In [113]:
# Jurabek Karimov
df_male[df_male["surname"] == "Karimov"].value_counts()

Unnamed: 0  name      surname  fullname          gender
27284       Djurabek  Karimov  Djurabek Karimov  M         1
29963       Ravil     Karimov  Ravil Karimov     M         1
40756       Eugene    Karimov  Eugene Karimov    M         1
52327       Adil      Karimov  Adil Karimov      M         1
53486       Iskandar  Karimov  Iskandar Karimov  M         1
dtype: int64

the correct fullname is  Jurabek Karimov, need to fix the male players dataset

In [114]:
replace_gender("Jurabek Karimov","M")

In [115]:
# Khumoun Sultanov
df_male[df_male["surname"] == "Sultanov"].value_counts()

Unnamed: 0  name      surname   fullname           gender
26923       Khumoyun  Sultanov  Khumoyun Sultanov  M         1
dtype: int64

the correct fullname is Khumoyun  Sultanov, need to fix the matches dataset

In [116]:
replace_name("Khumoun Sultanov","Khumoyun Sultanov","M")

In [117]:
# Kuan Yi Lee 
df_male[df_male["name"] == "Kuan"].value_counts()

Unnamed: 0  name  surname  fullname  gender
6161        Kuan  Lee      Kuan Lee  M         1
dtype: int64

the correct fullname is Kuan-yi Lee, need to fix the male players dataset

In [118]:
replace_gender("Kuan Yi Lee","M")

In [119]:
# Lloyd Harris
df_male[df_male["name"] == "Lloyd"].value_counts()

Unnamed: 0  name   surname        fullname             gender
564         Lloyd  Bourne         Lloyd Bourne         M         1
6251        Lloyd  Glasspool      Lloyd Glasspool      M         1
15013       Lloyd  Moglen         Lloyd Moglen         M         1
15518       Lloyd  Nordstrom      Lloyd Nordstrom      M         1
16508       Lloyd  Hartel         Lloyd Hartel         M         1
24147       Lloyd  Segal          Lloyd Segal          M         1
34116       Lloyd  Bruce Burgess  Lloyd Bruce Burgess  M         1
42065       Lloyd  Bartis         Lloyd Bartis         M         1
dtype: int64

In [ ]:
replace_gender("Yang Liu","M")

In [120]:
df_male[df_male["surname"] == "Harris"].value_counts()

Unnamed: 0  name                   surname  fullname                      gender
290         Ian                    Harris   Ian Harris                    M         1
1934        David                  Harris   David Harris                  M         1
48914       Brian                  Harris   Brian Harris                  M         1
47655       G                      Harris   G Harris                      M         1
44734       Lloyd George Muirhead  Harris   Lloyd George Muirhead Harris  M         1
42466       Glenford               Harris   Glenford Harris               M         1
39508       Martyn                 Harris   Martyn Harris                 M         1
39046       Dean                   Harris   Dean Harris                   M         1
37107       Jason                  Harris   Jason Harris                  M         1
34244       Patrick                Harris   Patrick Harris                M         1
32061       Christian              Harris   Christian Harri

the correct fullname is "Lloyd George Muirhead  Harris" but into the matches dataset there is only the first name

In [121]:
replace_name("Lloyd Harris","Lloyd George Muirhead Harris","M")

In [122]:
# Mackenzie McDonald 
df_male[df_male["surname"] == "McDonald"].value_counts()

Series([], dtype: int64)

In [123]:
df_male[df_male["name"] == "Mackenzie"].value_counts()

Unnamed: 0  name       surname   fullname            gender
11440       Mackenzie  Mcdonald  Mackenzie Mcdonald  M         1
38027       Mackenzie  Stearns   Mackenzie Stearns   M         1
dtype: int64

Mcdonald into the male players dataset doesn't has a capital letter. Need to fix it

In [124]:
replace_gender("Mackenzie McDonald","M")

In [125]:
# Pedro Martinez
print(df_male[df_male["surname"] == "Martinez"]["name"].unique())

['Geoff' 'Mario' 'Gerardo' 'Ignacio' 'Bernardo' 'Oscar' 'Eduardo'
 'Stephane' 'Pablo' 'Luis David' 'Mateo Nicolas' 'Angel' 'Borja' 'Jeff'
 'Octavio' 'Sandor' 'Emilio' 'Ivan' 'Erick' 'Aurelio' 'Jacobo' 'Miguel'
 'Rafael' 'Sergio' 'Carlos' 'Alvaro' 'Ricardo' 'Gilberto' 'Michael'
 'Victor' 'Jose Luis' 'Ivan Feliipe' 'Hugo' 'Santiago' 'Raul' 'Arturo'
 'Christian' 'Benjamin' 'Giovanni' 'Lucas Gabriel' 'Tomas Salvador'
 'Cristobal' 'Mario Roman' 'Douglas' 'Mael' 'Rolando' 'Erik' 'Adrian'
 'Juan' 'Fernando' 'Matias' 'Francisco' 'Nicolas' 'Mirko' 'Brian' 'Louroi'
 'T' 'Cesar' 'Javier' 'Alejandro' 'Flavio' 'Alex' 'Denilson' 'Mikel'
 'Antonio']


In [126]:
print(df_male[df_male["name"] == "Pedro"]["surname"].unique())

['Gonzalez' 'Rebolledo' 'Alatorre' 'Leon' 'Braga' 'Escudero' 'Nieto'
 'Canovas' 'Rico' 'Leao' 'Clar Rossello' 'Graber Anguita' 'Sousa'
 'Feitosa' 'Zerbini' 'Bernardi' 'Sakamoto' 'Dumont' 'Cachin' 'Cordeiro'
 'Mari' 'Lopez' 'Zannoni' 'Guimaraes' 'Alves' 'Campos' 'Ast' 'Araujo'
 'Rodrigues' 'Langre' 'Loewe' 'Garriga Nogues' 'Bueno Neto' 'Rosello'
 'Castella' 'Vega' 'Villena Martinez' 'Navedo Garcia' 'Pereira' 'Caniza'
 'Genovese' 'Gedda' 'Silva' 'Salas Lozano' 'Cacao' 'Zanotelli' 'Posselt'
 'Villar Almiron' 'Scocuglia' 'Rogar' 'Davisson' 'Freitas' 'Dalfre'
 'Verdugo' 'Ortega' 'De Paula' 'Bosio' 'Ecenarro Del Rio' 'Menna Barreto'
 'Machado' 'Custodio' 'Fontanet' 'Escudero Araujo' 'Duarte' 'Mejia'
 'Dominguez Alonso' 'Martinez Portero' 'Cots' 'Bardaji' 'Mugica'
 'Iamachkine' 'Mol' 'Munafo' 'Pontes' 'Nolasco Giangrecco' 'Lima Wagner'
 'Toccafondo' 'Almeida' 'Osete Montoro' 'Torralbo Munoz' 'Canavati'
 'Manso' 'Alonso Lopez' 'Ribeiro' 'Carvalho' 'Estevao'
 'Oranges De Figueiredo' 'Quijada' '

there is not a Pedro Martinez, but exists in reality, we could do data integration

In [127]:
replace_gender("Pedro Martinez","M")

In [128]:
# Sam Groth
df_male[df_male["surname"] == "Groth"].value_counts()

Unnamed: 0  name    surname  fullname      gender
5024        Samuel  Groth    Samuel Groth  M         1
7929        Paul    Groth    Paul Groth    M         1
44136       Dirk    Groth    Dirk Groth    M         1
dtype: int64

the real and correct fullname is SAMUEL GROTH, need to fix the matches dataset

In [129]:
replace_name("Sam Groth","Samuel Groth","M")

In [130]:
# Stan Wawrinka
df_male[df_male["surname"] == "Wawrinka"].value_counts()

Unnamed: 0  name       surname   fullname            gender
4525        Stanislas  Wawrinka  Stanislas Wawrinka  M         1
7393        Jonathan   Wawrinka  Jonathan Wawrinka   M         1
dtype: int64

the correct fullname is Stan Wawrinka, need to fix the male players dataset

In [131]:
replace_gender("Stan Wawrinka","M")

In [132]:
# Taylor Fritz
df_male[df_male["surname"] == "Fritz"].value_counts()

Unnamed: 0  name          surname  fullname            gender
229         Welry         Fritz    Welry Fritz         M         1
313         Bernard       Fritz    Bernard Fritz       M         1
6207        Felipe        Fritz    Felipe Fritz        M         1
7142        Guy           Fritz    Guy Fritz           M         1
9856        Harry         Fritz    Harry Fritz         M         1
26187       Taylor Harry  Fritz    Taylor Harry Fritz  M         1
40189       Julien        Fritz    Julien Fritz        M         1
50210       Thomas        Fritz    Thomas Fritz        M         1
dtype: int64

the complete fullname is "Taylor Harry  Fritz" but into the matches dataset there is only the first name

In [133]:
replace_name("Taylor Fritz","Taylor Harry Fritz","M")

In [134]:
# Yi Liu
df_players[df_players["fullname"] == "Yi Liu"].value_counts()

fullname  gender
Yi Liu    F         1
dtype: int64

In [135]:
replace_gender("Yi Liu","F")

In [136]:
#'Zeynep  Sena Sarioglan'
df_female[df_female["surname"] == "Sarioglan"].value_counts()

Unnamed: 0  name         surname    fullname               gender
16570       Zeynep Sena  Sarioglan  Zeynep Sena Sarioglan  F         1
dtype: int64

the correct fullname is Zeynep Sena Sariglan, need to fix the matches dataset if we want to keep also the second name

In [137]:
replace_name("Zeynep  Sena Sarioglan","Zeynep Sena Sarioglan","F")

In [138]:
# Ziyu Zhang
df_players[df_players["fullname"] == "Ziyu Zhang"].value_counts()

fullname    gender
Ziyu Zhang  F         1
dtype: int64

In [139]:
replace_gender("Ziyu Zhang","F")

In [140]:
# Jonathan Mridha
df_male[df_male["surname"] == "Mridha"].value_counts()

Series([], dtype: int64)

In [141]:
print(sorted(df_male[df_male["name"] == "Jonathan"]["surname"].unique()))

['Abadie', 'Andress', 'Arpin', 'Askey', 'Auer', 'Austin', 'Backman', 'Baker', 'Banos', 'Barrett', 'Beardsley', 'Beaskoetxea Etxabarr', 'Bejar', 'Bernay', 'Binding', 'Britt', 'Brooklyn', 'Calienes', 'Canter', 'Cardi', 'Chang', 'Chu', 'Clarke', 'Cornish', 'Dahan', 'Dasnieres De Veigy', 'David', 'Delaura', 'Delgado', 'Diomedi', 'Dipierro', 'Edmunds', 'Edwards', 'Engel', 'Erlich', 'Espinola', 'Eysseric', 'Florez Cohen', 'Ganger', 'Garcia Leo', 'Gasawneh', 'Glover', 'Goitia', 'Gomez', 'Gomont', 'Gonzalia', 'Gray', 'Greczula', 'Grolaux', 'Guatteri', 'Gutierrez', 'Hamilton', 'Hefetz', 'Hilaire', 'Ho', 'Hocking', 'Hooper', 'Howard', 'Howell', 'Hultkrantz Chesneau', 'Igbinovia', 'Jaklitsch', 'Janda', 'Jenkin', 'Jeuch', 'John', 'Kanar', 'Kaufman', 'Kazarian', 'Kessler', 'Kidane', 'Kinsella', 'Klein', 'Krimotat', 'Kuhn', 'Labella', 'Laubut', 'Leach', 'Lewis', 'Lim', 'Lints', 'Lynch', 'Magadan', 'Marray', 'Martinez Galvan', 'Maskens', 'Miller', 'Milsztajn', 'Molina Galvan', 'Morales', 'Morgan', 'M

In [142]:
replace_gender("Jonathan Mridha","M")

In [143]:
df[df["gender_w"].isnull()]["winner_name"].value_counts().to_dict()

{}

DONE ✅

## Profiles Dataset creation

Creating a set of attributes for our profiles' dataset.

In particular we are going to create the following attributes:
- fullname
- gender
- birth: year of player's birth
- hand: mode of the used hand in matches
- height: last known height of the player
- won_matches: number of matches won by the player
- lost_matches: number of matches lost by the player
- country: nationality of the player
- minutes: average played minutes per match
- rank: average rank position
- year: last match's year
- bp_save_ratio: avarage ratio of saved breakpoints (bp_saved / bp_faced)
- svpt_won_perc: avarage number of served points won (1stWon + 2stWon) / svpt  
- aces: average aces scored per match
- df_perc: avarage double faults made df / svpt
- matches: number of matches played
- percent_won: won matches percentage
- best_season: season (Summer/Winter) in which the player won more matches
- best_year: year in which the player won more matches
- best_surface: surface on which the player won more matches


fullname	gender	age	hand	tourney_spectators	won_matches	lost_matches	country	minutes	rank	year	bp_save_ratio	svpt_won_perc	ace	df_perc 	matches	best_season	best_year	best_surface

Preparing a new dataset

In [144]:
# duplicating matches' rows to have just one player per row (one row for the winner, one row for the loser)
to_keep = ["fullname", "gender", "height", "minutes","hand","tourney_spectators", "tourney_date", "country", "rank", "bpFaced", "bpSaved", "surface", "season", "1stWon", "2ndWon", "svpt", "ace", "df", "age"]
df_matches = df.rename(columns={"winner_name":"fullname", "gender_w":"gender", "winner_ht":"height", "winner_hand":"hand", "winner_ioc":"country", "winner_rank":"rank", "w_bpFaced":"bpFaced", "w_bpSaved":"bpSaved", "w_bpSaved":"bpSaved", "w_1stWon": "1stWon", "w_2ndWon": "2ndWon", "w_svpt":"svpt", "w_ace": "ace", "w_df":"df", "winner_age":"age"})[to_keep]
df_matches["won_matches"] = 1
df_matches["lost_matches"] = 0



df_matches2 = df.rename(columns={"loser_name":"fullname", "gender_l":"gender", "loser_ht":"height", "loser_hand":"hand", "loser_ioc":"country", "loser_rank":"rank", "l_bpFaced":"bpFaced", "l_bpSaved":"bpSaved", "l_bpSaved":"bpSaved", "l_1stWon": "1stWon", "l_2ndWon": "2ndWon", "l_svpt":"svpt", "l_ace":"ace", "l_df":"df", "loser_age":"age"})[to_keep]
df_matches2["won_matches"] = 0
df_matches2["lost_matches"] = 1

df_matches = pd.concat([df_matches, df_matches2]).sort_values(['fullname','tourney_date'])

#creating attributes
df_matches["year"] = (df_matches["tourney_date"]/10000).round()
df_matches["bp_save_ratio"] = df_matches["bpSaved"] / df_matches["bpFaced"]
df_matches["svpt_won_perc"] = (df_matches["1stWon"]+df_matches["2ndWon"]) / df_matches["svpt"]
df_matches["df_perc"] = df_matches["df"]/df_matches["svpt"]
df_matches["birth"] = df_matches["year"]-(df_matches["age"].round())
df_matches.drop(columns=["tourney_date", "1stWon", "2ndWon", "svpt", "bpFaced", "bpSaved"], inplace=True)

del df_matches2
print("Done")

Done


In [145]:
df_matches["lost_matches"].value_counts()

1    185799
0    185799
Name: lost_matches, dtype: int64

Creating profiles with basic statistics

In [146]:
#group by fullname to do statistics
df_matches["matches"] = 1
group = df_matches.groupby(["fullname"])
group = group.agg({"gender": pd.Series.mode,
                   "birth": pd.Series.mean,
                   "hand":pd.Series.mode,
                   "tourney_spectators":pd.Series.mean,
                   "won_matches": "sum",
                   "lost_matches": "sum",
                   "country": pd.Series.mode,
                   "minutes": pd.Series.mean,
                   "rank": pd.Series.mean,
                   "year": "last",
                   "bp_save_ratio": pd.Series.mean,
                   "svpt_won_perc": pd.Series.mean,
                   "ace": pd.Series.mean,
                   "df_perc": pd.Series.mean,
                   "matches": "sum"
                  })

group["birth"] = group["birth"].round()
group = group.rename({"year":"last_year", "rank":"avg_rank", "tourney_spectators": "avg_spectators", "minutes": "avg_minutes", "bp_save_ratio":"avg_bp_save_ratio", "svpt_won_perc":"avg_svpt_won_perc", "ace":"avg_ace", "df_perc":"avg_df_perc"})
print("Created basic statistics!")


Created basic statistics!


Creating "best of" statistics

In [ ]:
group.loc[group["hand"].str.len() != 1, "hand"] = "U"

In [147]:
#function that given a column name create a "best_<col_name>" column
#with the value that appears on most victories
def best_of(attr):
    group2 = df_matches.groupby(["fullname", attr]) #groupby fullname and attr
    group2 = group2.agg({"won_matches": "sum"})
    group2 = group2[group2["won_matches"] != 0] #removing row with no victories
    group2 = group2.loc[group2.groupby(level='fullname')['won_matches'].idxmax()].index.to_frame(index=False) #get the value of attr that has max hasWon
    group2.rename(columns={attr:"best_"+attr}, inplace=True)
    print("Created best_"+attr+"!")
    return group2

#creating all best_of
group = pd.merge(group, best_of("season"), how="left", on="fullname")
group = pd.merge(group, best_of("year"), how="left", on="fullname")
group = pd.merge(group, best_of("surface"), how="left", on="fullname")
group

Created best_season!
Created best_year!
Created best_surface!


,fullname,gender,age,hand,tourney_spectators,won_matches,lost_matches,country,minutes,rank,year,bp_save_ratio,svpt_won_perc,ace,df_perc,matches,best_season,best_year,best_surface
0,Aada Inna,F,NaN,U,4059.000000,0,1,FIN,NaN,NaN,2021.0,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
1,Aalisha Alexis,F,NaN,U,3237.500000,0,2,TTO,NaN,NaN,2019.0,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN
2,Aaliya Ebrahim,F,NaN,R,3632.666667,2,7,IND,NaN,NaN,2020.0,NaN,NaN,NaN,NaN,9,Summer,2019.0,Clay
3,Aaliyah Hohmann,F,NaN,U,2792.666667,1,2,DEU,NaN,NaN,2021.0,NaN,NaN,NaN,NaN,3,Summer,2021.0,Hard
4,Aalyka Ebrahim,F,NaN,U,3207.750000,0,4,IND,NaN,NaN,2019.0,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10095,Zuzana Zalabska,F,1985.0,R,1303.000000,2,1,CZE,NaN,858.000000,2016.0,NaN,NaN,NaN,NaN,3,Summer,2016.0,Clay
10096,Zuzana Zlochova,F,1990.0,R,3236.458101,93,86,SVK,90.571429,460.664804,2021.0,0.411585,0.453745,0.571429,0.073615,179,Summer,2016.0,Hard
10097,Zuzanna Bednarz,F,NaN,U,2206.500000,0,2,POL,NaN,NaN,2021.0,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN
10098,Zuzanna Szczepanska,F,NaN,R,2785.333333,0,3,POL,NaN,NaN,2021.0,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN


*the NaN in the best_** *field means that the player never won any match so it's not possible to compute a best of value* :

In [148]:
df_matches[df_matches["fullname"]=="Aada Inna"]

,fullname,gender,height,minutes,hand,tourney_spectators,country,rank,surface,season,ace,df,age,won_matches,lost_matches,year,bp_save_ratio,svpt_won_perc,df_perc,matches
167054,Aada Inna,F,NaN,NaN,U,4059.0,FIN,NaN,Clay,Summer,NaN,NaN,NaN,0,1,2021.0,NaN,NaN,NaN,1


In [149]:
group.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10100 entries, 0 to 10099
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   fullname            10100 non-null  object 
 1   gender              10100 non-null  object 
 2   age                 8009 non-null   float64
 3   hand                10100 non-null  object 
 4   tourney_spectators  10100 non-null  float64
 5   won_matches         10100 non-null  int64  
 6   lost_matches        10100 non-null  int64  
 7   country             10100 non-null  object 
 8   minutes             4236 non-null   float64
 9   rank                4445 non-null   float64
 10  year                10100 non-null  float64
 11  bp_save_ratio       4245 non-null   float64
 12  svpt_won_perc       4252 non-null   float64
 13  ace                 4253 non-null   float64
 14  df_perc             4252 non-null   float64
 15  matches             10100 non-null  int64  
 16  best

Percentage won attribute

In [150]:
group["percent_won"] = (group["won_matches"] * 100)/(group["won_matches"]+group["lost_matches"])
group["percent_won"] = group["percent_won"].astype(int)

In [ ]:
group.head()
group["hand"].value_counts()

In [ ]:
group.to_csv("profiles.csv")